## Big Spender Porfolio Calculator

Allocate a minimal budget that buys at least one share of each ticker and produces as-even-as-possible dollar weights across tickers.

In [ ]:
from typing import List, Optional, Tuple, Dict


def allocate_min_budget_even_weights(
    prices: List[float],
    tickers: Optional[List[str]] = None,
    *,
    max_iters: int = 10000,
    tiny: float = 1e-12,
) -> Tuple[Dict[str, int], float, float]:
    """Compute a minimal budget allocation that buys at least one share of
    each ticker and produces as-even-as-possible dollar weights across tickers.

    Algorithm (greedy, incremental):
    - Start with one share of each stock (minimal requirement). Spent = sum(prices).
    - Compute imbalance metric = sum(|weight_i - mean_weight|).
    - Repeatedly consider adding one share to each stock and pick the single-share
      purchase that yields the largest reduction in imbalance per dollar spent.
    - Stop when no single-share purchase reduces the imbalance.

    This yields a minimal extra spend (greedy heuristic) that improves balance
    until no single increment helps.
    Returns (allocations, spent, leftover) where leftover is 0.0 because we count
    the budget as the exact amount spent to reach the allocation.
    """
    if not prices:
        return ({}, 0.0, 0.0)

    n = len(prices)
    if tickers is None:
        keys = [str(i) for i in range(n)]
    else:
        if len(tickers) != n:
            raise ValueError('tickers length must match prices length')
        keys = list(tickers)

    qtys = [1 for _ in prices]
    spent = sum(float(p) for p in prices)

    def imbalance(qtys_local: List[int], spent_local: float) -> float:
        weights = [q * p for q, p in zip(qtys_local, prices)]
        mean = spent_local / n
        return sum(abs(w - mean) for w in weights)

    cur_metric = imbalance(qtys, spent)

    it = 0
    while it < max_iters:
        it += 1
        best = None  # (score, index, new_metric)
        for i in range(n):
            price = float(prices[i])
            new_spent = spent + price
            # weights after adding one share to i
            new_weights = [q * p for q, p in zip(qtys, prices)]
            new_weights[i] += price
            new_mean = new_spent / n
            new_metric = sum(abs(w - new_mean) for w in new_weights)
            improvement = cur_metric - new_metric
            if improvement > tiny:
                score = improvement / price
                if best is None or score > best[0]:
                    best = (score, i, new_metric)
        if best is None:
            break
        _, idx, new_metric = best
        qtys[idx] += 1
        spent += float(prices[idx])
        cur_metric = new_metric

    allocations = {k: q for k, q in zip(keys, qtys)}
    # We treat the spent amount as the effective budget (no leftover)
    return allocations, round(spent, 10), 0.0


# Example usage
tickers = ['ACN','CMG','DHR','ODFL','UPS','GIS']
prices = [241.24, 39.76, 192.94, 141.79, 84.18, 49.18]

alloc_min, spent_min, leftover_min = allocate_min_budget_even_weights(prices, tickers)
print('Greedy minimal allocation (one+ even weights):')
print('Allocations:', alloc_min)
print('Spent: USD', spent_min)


Greedy minimal allocation (one+ even weights):
Allocations: {'ACN': 1, 'CMG': 5, 'DHR': 1, 'ODFL': 1, 'UPS': 2, 'GIS': 4}
Spent: USD 1139.85
